# Visualizing arbitrary tract-level data

The following demonstrates how to visualize data that can be expressed as one number per tract, using the geoviews/holoviews stack.

At present, geoviews/geopandas are not available on the LSP, so running this there (even after pip installing geopandas and geoviews) crashes the notebook kernel.  But the following works on my laptop where the geoviews/holoviews stack is installed.

In [ ]:
import holoviews as hv
from cartopy import crs
import geopandas as gpd
hv.extension('bokeh')

In [ ]:
import numpy as np
import pandas as pd

# Random arbitrary data, indexed by tract
tracts = list(range(8000, 10000))
data = {'metric_1': np.random.random(len(tracts)),
        'metric_2': np.random.random(len(tracts))}
df = pd.DataFrame(index=tracts, data=data)

df.head()

In [ ]:
skymap = gpd.read_file('https://github.com/Quansight/lsst_dashboard/raw/master/deepCoadd_skyMap.geojson')
skymap.head()

In [ ]:
gdf = skymap.join(df, how='right')
gdf['tract'] = gdf.index
gdf.head()

In [ ]:
def create_dict(row):
    """create a dictionary representation of a df row"""
    d = {('x','y'): np.array(row.geometry.exterior.coords),
         'metric_1': row['metric_1'],
         'metric_2': row['metric_2'],
         'tract': row['tract'],
        }
    return d

# create a dictionary reprresentation of the dataframe
data = list(gdf.apply(create_dict, axis=1))

In [ ]:
hv.Polygons(data, vdims=['metric_1', 'metric_2','tract']).opts(tools=['hover'], width=1200, line_width=0, colorbar=True)